<a href="https://colab.research.google.com/github/Chiamakac/TRAININGS/blob/main/IgboBERT%202.0/Tuned_Copy_of_xlm_roberta_base_with%20merged%20dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# distilbert-base-uncased Token classification (PyTorch)

Install the Transformers and Datasets libraries to run this notebook.

In [ ]:
!pip install datasets transformers[sentencepiece]
#!pip install accelerate
# To run the training on TPU, you will need to uncomment the followin line:
# !pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!apt install git-lfs

     |████████████████████████████████| 306 kB 6.7 MB/s 
     |████████████████████████████████| 3.4 MB 46.7 MB/s 
     |████████████████████████████████| 133 kB 51.4 MB/s 
     |████████████████████████████████| 1.1 MB 31.8 MB/s 
     |████████████████████████████████| 243 kB 54.0 MB/s 
     |████████████████████████████████| 67 kB 5.4 MB/s 
     |████████████████████████████████| 192 kB 54.4 MB/s 
     |████████████████████████████████| 160 kB 52.9 MB/s 
     |████████████████████████████████| 271 kB 50.7 MB/s 
     |████████████████████████████████| 3.3 MB 34.9 MB/s 
     |████████████████████████████████| 596 kB 48.4 MB/s 
     |████████████████████████████████| 895 kB 42.7 MB/s 
     |████████████████████████████████| 1.2 MB 43.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The 

You will need to setup git, adapt your email and name in the following cell.

In [ ]:
!git config --global user.email "chukwunekechiamaka3@gmail.com"
!git config --global user.name "chiamaka"

You will also need to be logged in to the Hugging Face Hub. Execute the following and enter your credentials.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset('masakhaner', 'ibo')

Downloading:   0%|          | 0.00/2.75k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.25k [00:00<?, ?B/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/88.5k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/14.4k [00:00<?, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset masakhaner downloaded and prepared to /root/.cache/huggingface/datasets/masakhaner/ibo/1.0.0/e61b24903076a3af7682855beebb820ec64edad0d6787b148c473694592d10b3. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 2235
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 320
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 638
    })
})

In [ ]:
raw_datasets["train"][0]["tokens"]

['Ike',
 'ịda',
 'jụụ',
 'otụ',
 'nkeji',
 'banyere',
 'oke',
 'ogbugbu',
 'na',
 '-',
 'eme',
 "n'ala",
 'Naijiria',
 'agwụla',
 'Ekweremmadụ']

In [ ]:
raw_datasets["train"][0]["ner_tags"]

[0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 5, 0, 1]

In [ ]:
ner_feature = raw_datasets["train"].features["ner_tags"]
ner_feature

Sequence(feature=ClassLabel(num_classes=9, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-DATE', 'I-DATE'], names_file=None, id=None), length=-1, id=None)

In [ ]:
label_names = ner_feature.feature.names
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-DATE', 'I-DATE']

In [ ]:
words = raw_datasets["train"][0]["tokens"]
labels = raw_datasets["train"][0]["ner_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

Ike ịda jụụ otụ    nkeji  banyere oke ogbugbu na - eme n'ala Naijiria agwụla Ekweremmadụ 
O   O   O   B-DATE I-DATE O       O   O       O  O O   O     B-LOC    O      B-PER       


In [ ]:
from transformers import AutoTokenizer

model_checkpoint ="xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/512 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

In [ ]:
tokenizer.is_fast

True

In [ ]:
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
inputs.tokens()

['<s>',
 '▁I',
 'ke',
 '▁',
 'ị',
 'da',
 '▁j',
 'ụ',
 'ụ',
 '▁o',
 't',
 'ụ',
 '▁',
 'nke',
 'ji',
 '▁bany',
 'ere',
 '▁o',
 'ke',
 '▁og',
 'bug',
 'bu',
 '▁na',
 '▁-',
 '▁em',
 'e',
 '▁n',
 "'",
 'ala',
 '▁Na',
 'iji',
 'ria',
 '▁ag',
 'w',
 'ụ',
 'la',
 '▁Ek',
 'wer',
 'em',
 'mad',
 'ụ',
 '</s>']

In [ ]:
inputs.word_ids()

[None,
 0,
 0,
 1,
 1,
 1,
 2,
 2,
 2,
 3,
 3,
 3,
 4,
 4,
 4,
 5,
 5,
 6,
 6,
 7,
 7,
 7,
 8,
 9,
 10,
 10,
 11,
 11,
 11,
 12,
 12,
 12,
 13,
 13,
 13,
 13,
 14,
 14,
 14,
 14,
 14,
 None]

In [ ]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [ ]:
labels = raw_datasets["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[0, 0, 0, 7, 8, 0, 0, 0, 0, 0, 0, 0, 5, 0, 1]
[-100, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 8, 8, 8, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 6, 6, 0, 0, 0, 0, 1, 2, 2, 2, 2, -100]


In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [ ]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [ ]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

tensor([[-100,    0,    0,    0,    0,    0,    0,    0,    0,    7,    8,    8,
            8,    8,    8,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    5,    6,    6,    0,    0,    0,    0,
            1,    2,    2,    2,    2, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100],
        [-100,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    5,    6,    6,    6,    6,    0,    0,    1,    2,
            2,    2,    2,    2,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    7,
            8,    8,    8,    8,    0,    0,    0,    0,    0, 

In [ ]:
for i in range(2):
    print(tokenized_datasets["train"][i]["labels"])

[-100, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 8, 8, 8, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 6, 6, 0, 0, 0, 0, 1, 2, 2, 2, 2, -100]
[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 6, 6, 6, 6, 0, 0, 1, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 8, 8, 8, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 6, 6, 6, 6, 0, 0, 0, 0, 0, 0, 0, 0, -100]


In [ ]:
!pip install seqeval

     |████████████████████████████████| 43 kB 1.2 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=a437519cbefa3ade15ac2389681514cb820a69da0d19bfe1879a56770e8593a0
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


In [ ]:
from datasets import load_metric

metric = load_metric("seqeval")

Downloading:   0%|          | 0.00/2.48k [00:00<?, ?B/s]

In [ ]:
labels = raw_datasets["train"][0]["ner_tags"]
labels = [label_names[i] for i in labels]
labels

['O',
 'O',
 'O',
 'B-DATE',
 'I-DATE',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-LOC',
 'O',
 'B-PER']

In [ ]:
predictions = labels.copy()
predictions[2] = "O"
metric.compute(predictions=[predictions], references=[labels])

{'DATE': {'f1': 1.0, 'number': 1, 'precision': 1.0, 'recall': 1.0},
 'LOC': {'f1': 1.0, 'number': 1, 'precision': 1.0, 'recall': 1.0},
 'PER': {'f1': 1.0, 'number': 1, 'precision': 1.0, 'recall': 1.0},
 'overall_accuracy': 1.0,
 'overall_f1': 1.0,
 'overall_precision': 1.0,
 'overall_recall': 1.0}

In [ ]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [ ]:
id2label = {str(i): label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [ ]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

Downloading:   0%|          | 0.00/1.04G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForTokenClassification: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-st

In [ ]:
model.config.num_labels

9

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    "xlm-roberta-base-ner",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-4,
    num_train_epochs=20,
    weight_decay=0.01,
    #push_to_hub=True,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

***** Running training *****
  Num examples = 2235
  Num Epochs = 20
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 5600


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.167040,0.678195,0.751667,0.713043,0.955010
2,0.213700,0.126298,0.726316,0.805000,0.763636,0.963842
3,0.213700,0.134127,0.726361,0.845000,0.781202,0.966464
4,0.076800,0.130057,0.753383,0.835000,0.792095,0.962462
5,0.076800,0.142903,0.785484,0.811667,0.798361,0.970121
6,0.044800,0.126013,0.787167,0.838333,0.811945,0.969983
7,0.044800,0.127395,0.811802,0.848333,0.829666,0.973710
8,0.023800,0.143733,0.800933,0.858333,0.828640,0.973917
9,0.012900,0.163170,0.800000,0.866667,0.832000,0.973434
10,0.012900,0.152978,0.804207,0.828333,0.816092,0.972468


***** Running Evaluation *****
  Num examples = 320
  Batch size = 8
Saving model checkpoint to xlm-roberta-base-ner/checkpoint-280
Configuration saved in xlm-roberta-base-ner/checkpoint-280/config.json
Model weights saved in xlm-roberta-base-ner/checkpoint-280/pytorch_model.bin
tokenizer config file saved in xlm-roberta-base-ner/checkpoint-280/tokenizer_config.json
Special tokens file saved in xlm-roberta-base-ner/checkpoint-280/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 320
  Batch size = 8
Saving model checkpoint to xlm-roberta-base-ner/checkpoint-560
Configuration saved in xlm-roberta-base-ner/checkpoint-560/config.json
Model weights saved in xlm-roberta-base-ner/checkpoint-560/pytorch_model.bin
tokenizer config file saved in xlm-roberta-base-ner/checkpoint-560/tokenizer_config.json
Special tokens file saved in xlm-roberta-base-ner/checkpoint-560/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 320
  Batch size = 8
Saving model ch

TrainOutput(global_step=5600, training_loss=0.03537453521242631, metrics={'train_runtime': 1126.932, 'train_samples_per_second': 39.665, 'train_steps_per_second': 4.969, 'total_flos': 1960475422686876.0, 'train_loss': 0.03537453521242631, 'epoch': 20.0})

In [ ]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "/content/xlm-roberta-base-ner/checkpoint-5600"
token_classifier = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="simple"
)
token_classifier("Google chetara ụbọchị ọmụmụ Keshi mgbe Obasanjo chịrị Naịjirịa afọ asatọ na ọchịchị onye kwuo uche ya."
                 "Onyeisi ndị na-emenyu ọkụ na, Legọọsi steeti bụ Rasak Fadipe ekwuola na onweghi onye nwụrụ n'ime ọkụ ahụ gbara n'ehihie ụbọchị 24 Jenuwarị, 2018.")

loading configuration file /content/xlm-roberta-base-ner/checkpoint-5600/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "/content/xlm-roberta-base-ner/checkpoint-5600",
  "architectures": [
    "XLMRobertaForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-PER",
    "2": "I-PER",
    "3": "B-ORG",
    "4": "I-ORG",
    "5": "B-LOC",
    "6": "I-LOC",
    "7": "B-DATE",
    "8": "I-DATE"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-DATE": "7",
    "B-LOC": "5",
    "B-ORG": "3",
    "B-PER": "1",
    "I-DATE": "8",
    "I-LOC": "6",
    "I-ORG": "4",
    "I-PER": "2",
    "O": "0"
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 1

[{'end': 6,
  'entity_group': 'ORG',
  'score': 0.9998722,
  'start': 0,
  'word': 'Google'},
 {'end': 21,
  'entity_group': 'DATE',
  'score': 0.99929976,
  'start': 15,
  'word': 'ụbọchị'},
 {'end': 33,
  'entity_group': 'PER',
  'score': 0.99990076,
  'start': 28,
  'word': 'Keshi'},
 {'end': 47,
  'entity_group': 'PER',
  'score': 0.99992925,
  'start': 39,
  'word': 'Obasanjo'},
 {'end': 62,
  'entity_group': 'LOC',
  'score': 0.9999356,
  'start': 54,
  'word': 'Naịjirịa'},
 {'end': 72,
  'entity_group': 'DATE',
  'score': 0.99981546,
  'start': 63,
  'word': 'afọ asatọ'},
 {'end': 146,
  'entity_group': 'LOC',
  'score': 0.9993294,
  'start': 132,
  'word': 'Legọọsi steeti'},
 {'end': 162,
  'entity_group': 'PER',
  'score': 0.9999323,
  'start': 150,
  'word': 'Rasak Fadipe'},
 {'end': 247,
  'entity_group': 'DATE',
  'score': 0.99986255,
  'start': 213,
  'word': "n'ehihie ụbọchị 24 Jenuwarị, 2018."}]

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import shutil
shutil.move('/content/xlm-roberta-base-ner/checkpoint-5600','/content/gdrive/MyDrive/IBO_BETA/ LREC FINAL TRAINING')

'/content/gdrive/MyDrive/IBO_BETA/ LREC FINAL TRAINING/checkpoint-5600'